In [145]:
%matplotlib inline

import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array

In [146]:
labels = {"成熟": 0, "陰沈": 1, "搞怪": 2, "清純可愛": 3, "知性": 4, "陽光開朗": 5, "性感": 6, "老實": 7}
train_x = []
train_y = []

for i in labels.keys():
    path = f"/Volumes/FUJITSU/{i}" 
    files= os.listdir(path)
    print(i)
    for file in files:
        if not os.path.isdir(file):
            img = load_img(f"{path}/{file}")
            train_x.append(img_to_array(img))
            train_y.append(labels[i])
train_x = np.array(train_x)
train_y = np.array(train_y)

成熟
陰沈
搞怪
清純可愛
知性
陽光開朗
性感
老實


In [147]:
train_y

array([0, 0, 0, ..., 7, 7, 7])

In [148]:
from keras.utils import np_utils # one-hot encoding

train_y = np_utils.to_categorical(train_y, 8)
train_x = train_x / 255

In [149]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPool2D

In [150]:
face_cf = Sequential()

In [151]:
# conv --> maxpool --> conv --> maxpool --> conv --> maxpool
face_cf.add(Conv2D(2, (9, 9), padding="same", input_shape=(218, 178, 3), activation="relu"))
face_cf.add(MaxPool2D(pool_size=(3,3)))
face_cf.add(Conv2D(4, (9, 9), padding="same", activation="relu"))
face_cf.add(MaxPool2D(pool_size=(3,3)))
face_cf.add(Conv2D(8, (9, 9), padding="same", activation="relu"))
face_cf.add(MaxPool2D(pool_size=(3,3)))

# 進入 fully connected 前需要拉平
face_cf.add(Flatten())
face_cf.add(Dense(10, activation="relu"))
face_cf.add(Dense(20, activation="relu"))
face_cf.add(Dense(8, activation="softmax"))

In [152]:
face_cf.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 218, 178, 2)       488       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 72, 59, 2)         0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 72, 59, 4)         652       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 24, 19, 4)         0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 24, 19, 8)         2600      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 8, 6, 8)           0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 384)              

In [153]:
face_cf.compile(loss="mse", optimizer="adam", metrics=["accuracy"])

In [ ]:
face_cf.fit(train_x, train_y, batch_size=1000, epochs=10)

Epoch 1/10
2362/2362 [==============================] - 133s 57ms/step - loss: 0.1095 - accuracy: 0.0830
Epoch 2/10
2362/2362 [==============================] - 140s 59ms/step - loss: 0.1093 - accuracy: 0.1859
Epoch 3/10
2362/2362 [==============================] - 151s 64ms/step - loss: 0.1093 - accuracy: 0.2070
Epoch 4/10
2362/2362 [==============================] - 145s 62ms/step - loss: 0.1093 - accuracy: 0.2125
Epoch 5/10
2362/2362 [==============================] - 137s 58ms/step - loss: 0.1092 - accuracy: 0.2096
Epoch 6/10
2362/2362 [==============================] - 152s 64ms/step - loss: 0.1092 - accuracy: 0.2087
Epoch 7/10
2362/2362 [==============================] - 168s 71ms/step - loss: 0.1091 - accuracy: 0.2075
Epoch 8/10
2362/2362 [==============================] - 145s 61ms/step - loss: 0.1091 - accuracy: 0.2045
Epoch 9/10
1000/2362 [===========>..................] - ETA: 1:20 - loss: 0.1090 - accuracy: 0.2040